In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException


driver = webdriver.Chrome()
URL = "https://www.metacritic.com/browse/movies/score/metascore/year/filtered"
driver.get(URL)

delay = 10

print (driver)

time.sleep(1)

try: # if the reject cookies button is there then click it
    reject_cookies_btn = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-reject-all-handler"]')))
    reject_cookies_btn.click()

except NoSuchElementException: # open the cookies settings, then click reject
    cookies_settings_btn = driver.find_element(by=By.XPATH, value='//*[@id="ot-sdk-btn"]')
    cookies_settings_btn.click()
    reject_cookies_btn = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-pc-sdk"]/div[3]/div[1]/button[1]')))
    reject_cookies_btn.click()

except TimeoutException:
    print ("Loading cookies took too much time!")

except:
    pass


movies = driver.find_elements(By.XPATH, '//div[@class="title_bump"]//a[@class="title"]')
links = [movie.get_attribute('href') for movie in movies]
print("test!")
print(links)


<selenium.webdriver.chrome.webdriver.WebDriver (session="647af352397433c72025da4ca8c9df56")>
test!
['https://www.metacritic.com/movie/mr-bachmann-and-his-class', 'https://www.metacritic.com/movie/the-worst-person-in-the-world', 'https://www.metacritic.com/movie/hit-the-road', 'https://www.metacritic.com/movie/who-we-are-a-chronicle-of-racism-in-america', 'https://www.metacritic.com/movie/great-freedom', 'https://www.metacritic.com/movie/decision-to-leave', 'https://www.metacritic.com/movie/aftershock-2022', 'https://www.metacritic.com/movie/a-night-of-knowing-nothing', 'https://www.metacritic.com/movie/in-front-of-your-face', 'https://www.metacritic.com/movie/playground', 'https://www.metacritic.com/movie/happening', 'https://www.metacritic.com/movie/moonage-daydream', 'https://www.metacritic.com/movie/fire-of-love', 'https://www.metacritic.com/movie/the-fallout', 'https://www.metacritic.com/movie/rrr', 'https://www.metacritic.com/movie/lingui', 'https://www.metacritic.com/movie/the-gi